# convert station id coordinates from NAD83 to WGS84

In [9]:
import ogr, osr
import gmaps
import gmaps.datasets
import os
# Find codes in this file ~/anaconda3/envs/tf2/share/gdal/pcs.csv
# 
# Using google
# WGS 84 is defined under EPSG code 4326.
# WGS84 EPSG 32618  http://www.spatialreference.org/ref/epsg/wgs-84-utm-zone-18n/

# NAD83 - EPSG:4269 - EPSG.io  possibly wrong
# NAD83  32147 http://spatialreference.org/ref/epsg/32147/

##################3
# Using the pcs.csv file

# source geogcrs code
# 4269    NAD83 lots of these
# 4326    WGS 84 lots of these


In [10]:
# Spatial Reference System
#
# fail
#inputEPSG = 32147 # NAD83
#outputEPSG = 32618 # WGS84
# fail but close
#inputEPSG = 4269 # NAD83
#outputEPSG = 4326 # WGS84
# fail
inputEPSG = 4269 # NAD83
outputEPSG = 4326 # WGS84
#

In [11]:
# Site Number: 0204300267
# Site Name: BEGGARS BRIDGE CREEK NEAR DAWLEY CORNERS, VA
# Site Type: Stream
# Agency: USGS
# Latitude 36°40'47.3",   Longitude 75°59'02.4"   NAD83
# Virginia Beach City, Virginia, Hydrologic Unit 03010205
# Datum of gage: 0.00 feet above   NAVD88.

degrees = 36
minutes = 40.0
seconds = 47.3

# According to wikipedia I should 
# use floor division for minutes
pointX_floor = degrees + minutes//60.0 + seconds/3600.0 

# According to Tuthill, don't do this
pointX = degrees + minutes/60.0 + seconds/3600.0 


degrees = 75
minutes = 59.0
seconds = 02.4

# According to wikipedia I should 
# use floor division for minutes
pointY_floor = degrees + minutes//60.0 + seconds/3600.0 

# According to Tuthill, don't do this
pointY = degrees + minutes/60.0 + seconds/3600.0 

# since the number should be negative
pointY = -pointY

print(pointX, pointY)
print("using floor {}, {}".format(pointX_floor, pointY_floor))

# The non use of floor seems to work.  Also its good enough that
# the change is coordinate systems seems to not be required.

36.679805555555554 -75.984
using floor 36.01313888888889, 75.00066666666666


In [12]:
# create a geometry from coordinates
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(pointX, pointY)

# create coordinate transformation
inSpatialRef = osr.SpatialReference()
inSpatialRef.ImportFromEPSG(inputEPSG)

outSpatialRef = osr.SpatialReference()
outSpatialRef.ImportFromEPSG(outputEPSG)

coordTransform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)

# transform point
point.Transform(coordTransform)


print(point.GetX(), point.GetY())

# a correct answer will be something like
# 36.680784, -76.003623

36.679805555555554 -75.984


In [13]:
# fwiw, here is a guy using geopandas to convert data to NAD83 and he uses 4269
# https://stackoverflow.com/questions/41820969/python-geo-spatial-coordinate-format-conversion

In [14]:
# another variant.  Essentially the same.
from osgeo import osr
inp= osr.SpatialReference()
inp.ImportFromEPSG(inputEPSG)
out= osr.SpatialReference()
out.ImportFromEPSG(outputEPSG)
transformation = osr.CoordinateTransformation(inp,out)
print(transformation.TransformPoint(pointX,pointY))
#(-74.27000000011289, 40.46999999990432, 0.0)

(36.679805555555554, -75.984, 0.0)


In [15]:
gmaps.configure(api_key=os.environ["GOOGLE_MAPS_API_KEY"])

In [16]:
fig = gmaps.figure()
# print the two locations, correct without floor and incorrect with floor
locations = [(36.679805555555554, -75.984), (36.01313888888889, -75.00066666666666)]
fig = gmaps.figure(center=(36.0, -75.0), zoom_level=8)
fig = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
# give ablitly to turn on satelite
fig = gmaps.figure(map_type='HYBRID')

# show stations as green dots
stations_layer = gmaps.symbol_layer(
    locations, fill_color="green", stroke_color="green", scale=2
)
fig = gmaps.figure()
fig.add_layer(stations_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# converting all the sites to decimal lat/longs

In [31]:
loc_degminsec_dict = {"0204288721":{"lat":{"deg":36.0,"min":49.0, "sec":38.0},
                                    "lon":{"deg":-76.0,"min":9.0, "sec":56.0}},
                      
                      "0204291317":{"lat":{"deg":36.0,"min":50.0, "sec":35.9},
                                    "lon":{"deg":-76.0,"min":7.0, "sec":28.1}},
                      
                      "0204293125":{"lat":{"deg":36.0,"min":50.0, "sec":28.1},
                                    "lon":{"deg":-76.0,"min":3.0, "sec":25.7}},
                      
                      "0204295505":{"lat":{"deg":36.0,"min":51.0, "sec":33.4},
                                    "lon":{"deg":-75.0,"min":59, "sec":4.1}},
                      
                      "02042928":{"lat":{"deg":36.0,"min":48.0, "sec":56.8},
                                    "lon":{"deg":-76.0,"min":3.0, "sec":39.3}},
                      
                      "0204297575":{"lat":{"deg":36.0,"min":49.0, "sec":29.2},
                                    "lon":{"deg":-75.0,"min":59.0, "sec":6.9}},
                      
                      "02043155":{"lat":{"deg":36.0,"min":47.0, "sec":18.1},
                                    "lon":{"deg":-76.0,"min":4.0, "sec":14.8}},
                      
                      "0204288539":{"lat":{"deg":36.0,"min":42.0, "sec":42.1},
                                    "lon":{"deg":-76.0,"min":16.0, "sec":5.6}},
                      
                      "0204309906":{"lat":{"deg":36.0,"min":41.0, "sec":26.32},
                                    "lon":{"deg":-76.0,"min":12.0, "sec":44.34}},
                      
                      "0204300267":{"lat":{"deg":36.0,"min":40.0, "sec":47.3},
                                    "lon":{"deg":-75.0,"min":59, "sec":2.4}},
                     }

In [32]:
loc_degminsec_dict["0204288721"]['lat']

{'deg': 36.0, 'min': 49.0, 'sec': 38.0}

In [33]:
loc_degminsec_dict["0204288721"]['lat']['deg']

36.0

In [41]:
# convert stations location in degrees, minutes, seconds to degrees.fractional degrees
def convert_station_in_dms_to_dfd(station):
    degrees = loc_degminsec_dict[station]['lat']['deg']
    minutes = loc_degminsec_dict[station]['lat']['min']
    seconds = loc_degminsec_dict[station]['lat']['sec']
    pointX = degrees + minutes/60.0 + seconds/3600.0 
    
    degrees = loc_degminsec_dict[station]['lon']['deg']
    minutes = loc_degminsec_dict[station]['lon']['min']
    seconds = loc_degminsec_dict[station]['lon']['sec']    
    pointY = degrees + minutes/60.0 + seconds/3600.0 

    return pointX, pointY

In [42]:
pointX, pointY = convert_station_in_dms_to_dfd("0204288721")

In [43]:
(pointX, pointY)

(36.827222222222225, -75.83444444444444)

In [44]:
for station in loc_degminsec_dict:
    pointX, pointY = convert_station_in_dms_to_dfd(station)
    print("station id: {} lat: {} lon: {}".format(station, pointX, pointY))

station id: 0204288721 lat: 36.827222222222225 lon: -75.83444444444444
station id: 0204291317 lat: 36.84330555555556 lon: -75.87552777777779
station id: 0204293125 lat: 36.84113888888889 lon: -75.94286111111111
station id: 0204295505 lat: 36.85927777777778 lon: -74.01552777777778
station id: 02042928 lat: 36.815777777777775 lon: -75.93908333333334
station id: 0204297575 lat: 36.82477777777778 lon: -74.01474999999999
station id: 02043155 lat: 36.78836111111111 lon: -75.92922222222222
station id: 0204288539 lat: 36.71169444444445 lon: -75.73177777777778
station id: 0204309906 lat: 36.690644444444445 lon: -75.78768333333333
station id: 0204300267 lat: 36.679805555555554 lon: -74.016
